<a href="https://colab.research.google.com/github/Danish-Hussain-DSAI/Deep-Fake-on-RESNET50/blob/main/capstone_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#!7za -y x "/content/drive/MyDrive/ds fake and real images/real.zip"

In [3]:
#!7za -y x "/content/drive/MyDrive/ds fake and real images/fake.zip"

In [4]:
import numpy as np
import tensorflow as tf


In [5]:

folder_path="/content/sample_data/Images"
dataset=tf.keras.utils.image_dataset_from_directory(
    folder_path,
    image_size=(64,64),
    batch_size=16,







                                                    )
dataset_shape = dataset.element_spec[0].shape
print("Shape of the dataset:", dataset_shape)

Found 140000 files belonging to 2 classes.
Shape of the dataset: (None, 64, 64, 3)


In [6]:
classes=dataset.class_names
print(classes)


['fake', 'real']


In [7]:
from tensorflow.keras.layers import Lambda

normalize = Lambda(lambda x: x / 255.0)



dataset = dataset.map(
    lambda x, y: (normalize(x), y)
)



In [8]:
# Define the size of the training set (e.g., 80% for training)
train_size = 0.8

# Calculate the number of batches for training and testing
num_batches = len(dataset)
num_train_batches = int(train_size * num_batches)
num_test_batches = num_batches - num_train_batches

# Split the dataset into training and testing sets
train_dataset = dataset.take(num_train_batches)
test_dataset = dataset.skip(num_train_batches)

# Print the sizes of the training and testing sets
print(f"Training set size: {num_train_batches} batches")
print(f"Testing set size: {num_test_batches} batches")


Training set size: 7000 batches
Testing set size: 1750 batches


In [9]:
# one-hot encoding
train_dataset = train_dataset.map(lambda x, y: (x, tf.one_hot(y, depth=2)))
test_dataset = test_dataset.map(lambda x, y: (x, tf.one_hot(y, depth=2)))

In [10]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

# Define your data augmentation transformations
data_augmentation = tf.keras.Sequential([
    preprocessing.Rescaling(1./255),  # Rescale pixel values to [0, 1]
    preprocessing.RandomFlip("horizontal"),  # Randomly flip images horizontally
    preprocessing.RandomRotation(0.2),  # Randomly rotate images by up to 20 degrees
    preprocessing.RandomZoom(0.2),  # Randomly zoom images by up to 20%
    preprocessing.RandomContrast(0.2),  # Randomly adjust image contrast
])

# Apply data augmentation to your original dataset and create augmented dataset
augmented_dataset = train_dataset.map(lambda x, y: (data_augmentation(x, training=True), y))

# Concatenate the augmented data with the original dataset
combined_dataset = train_dataset.concatenate(augmented_dataset)

# Shuffle the combined dataset
train_dataset = combined_dataset.shuffle(buffer_size=1000)  # You can adjust the buffer_size as needed

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define your CNN model
model = Sequential()

# Add a Conv2D layer with 32 filters, a kernel size of (3, 3), and 'relu' activation
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))

# Add a MaxPooling2D layer to down-sample the spatial dimensions
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add another Conv2D layer with 64 filters and a kernel size of (3, 3)
model.add(Conv2D(64, (3, 3), activation='relu'))

# Add another MaxPooling2D layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output before feeding it into Dense layers
model.add(Flatten())

# Add Dense layers for classification
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='sigmoid'))  # Adjust 'num_classes' based on your task

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset)

Epoch 1/10
14000/14000 [==============================] - 628s 42ms/step - loss: 0.5833 - accuracy: 0.6352 - val_loss: 0.6914 - val_accuracy: 0.5494
Epoch 2/10
14000/14000 [==============================] - 545s 38ms/step - loss: 0.5038 - accuracy: 0.6869 - val_loss: 0.6649 - val_accuracy: 0.6797
Epoch 3/10
14000/14000 [==============================] - 547s 38ms/step - loss: 0.4474 - accuracy: 0.7427 - val_loss: 0.7505 - val_accuracy: 0.5808
Epoch 4/10
14000/14000 [==============================] - 608s 42ms/step - loss: 0.4019 - accuracy: 0.7809 - val_loss: 0.6091 - val_accuracy: 0.6805
Epoch 5/10
14000/14000 [==============================] - 606s 42ms/step - loss: 0.3726 - accuracy: 0.7982 - val_loss: 0.7022 - val_accuracy: 0.6027
Epoch 6/10
14000/14000 [==============================] - 607s 42ms/step - loss: 0.3519 - accuracy: 0.8109 - val_loss: 0.4810 - val_accuracy: 0.7873
Epoch 7/10
14000/14000 [==============================] - 606s 42ms/step - loss: 0.3386 - accuracy: 0.8194

In [13]:
model.save('model.h5') # Saves the entire model to a single artifact

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
